In [1]:
import pandas as pd

import os
import time
from collections import defaultdict

from dotenv import load_dotenv

import psycopg2
from psycopg2.extras import NamedTupleCursor

from pymongo import MongoClient

from neo4j import GraphDatabase

import solutionbase
from solutions import RelationalSolution, DocumentSolution, GraphSolution

In [2]:

load_dotenv();

In [3]:
relational_solution = RelationalSolution(psycopg2.connect(
    user = os.getenv("POSTGRES_USER"),
    password = os.getenv("POSTGRES_PASSWORD"),
    host = 'localhost',
    database = os.getenv("POSTGRES_DB"),
    cursor_factory=NamedTupleCursor,
))

document_solution = DocumentSolution(MongoClient('mongodb://localhost'))

graph_solution = GraphSolution(GraphDatabase.driver('bolt://localhost:7687', auth=('neo4j', 'bitnami1')))

all_solutions = [
    ('реляційна', relational_solution),
    ('документна', document_solution),
    ('графова', graph_solution),
]

In [4]:
def test_cases(sol: solutionbase.Solution):
    return [
        ('забрати рюзюме', lambda: sol.get_resume_for_user('User_1')),
        ('забрати всі хоббі які існують в резюме', lambda: sol.get_all_hobbies_from_resumes()),
        ('забрати всі міста, що зустрічаються в резюме', lambda: sol.get_all_cities_from_resumes()),
        ('забрати хоббі всіх здобувачів, що мешкають в заданому місті', lambda: sol.get_all_hobbies_of_users_from_city('City_1')),
        ('забрати всіх здобувачів, що працювали в одному закладі', lambda: sol.get_all_users_from_same_company()),
    ]

# Первіримо коректність запитів на даних малого обʼєму

In [5]:
generated_data_small = solutionbase.generate(size=5)
for _, solution in all_solutions:
    solution.populate(generated_data_small)

In [6]:
query_result_table = defaultdict(defaultdict)
for solution_name, solution in all_solutions:
    for test_name, test_func in test_cases(solution):
        query_result_table[solution_name][test_name] = test_func()

In [7]:
pd.DataFrame(query_result_table)

,реляційна,документна,графова
забрати рюзюме,"(1, 1, City_2, Company_1)","{'_id': 1, 'hobbies': ['Hobby_1', 'Hobby_3'], ...","(city, id)"
забрати всі хоббі які існують в резюме,"[(Hobby_0,), (Hobby_1,), (Hobby_2,), (Hobby_3,...","[Hobby_0, Hobby_1, Hobby_2, Hobby_3, Hobby_4]","[Hobby_0, Hobby_1, Hobby_2, Hobby_3, Hobby_4]"
"забрати всі міста, що зустрічаються в резюме","[City_0, City_1, City_2]","[City_0, City_1, City_2]","[City_0, City_2, City_1]"
"забрати хоббі всіх здобувачів, що мешкають в заданому місті","[Hobby_0, Hobby_2]","[Hobby_0, Hobby_2]","[Hobby_0, Hobby_2]"
"забрати всіх здобувачів, що працювали в одному закладі","[(User_1,), (User_2,), (User_3,), (User_4,)]","[User_1, User_2, User_3, User_4]","[User_2, User_1, User_4, User_3]"


## Виміряємо час запитів для сгенерованих запитів більшого розміру

In [8]:
generated_data_large = solutionbase.generate(size=1000)
for _, solution in all_solutions:
    solution.populate(generated_data_large)

In [9]:
timing_table = defaultdict(defaultdict)
for solution_name, solution in all_solutions:
    for test_name, test_func in test_cases(solution):
        start_time = time.time()
        test_func()
        end_time = time.time()
        timing_table[f"{solution_name} (мс)"][test_name] = (end_time - start_time) * 1000

In [10]:
pd.DataFrame(timing_table)

,реляційна (мс),документна (мс),графова (мс)
забрати рюзюме,3.072023,1.353979,9.654999
забрати всі хоббі які існують в резюме,3.918886,1.968861,9.696960
"забрати всі міста, що зустрічаються в резюме",1.372099,0.898123,6.970882
"забрати хоббі всіх здобувачів, що мешкають в заданому місті",0.912189,0.550985,11.950731
"забрати всіх здобувачів, що працювали в одному закладі",1.771927,6.048203,18.023014
